In [ ]:
import numpy as np
from iminuit import Minuit
import scipy.interpolate as interpolate
from EBL_MC_functions import * 
import matplotlib.pyplot as plt
import h5py
import uproot
from ebltable.tau_from_model import OptDepth
import os

In [ ]:
data_path = "data/Output_flute.root" #path of the data (dl3 file or Output_flute.root file)

#Source info
extratxt = "paper"
Systematics = False
sys_err = 0.07#0.0225

Source_name = '1ES1011_Feb2014' #source name

Emin, Emax = 0.06, 20. #maximum and minimum energy you want to take into account in TeV

betterEminmax = False
fit_func_name = "LP" #options are PWL, LP, freeLP and MBPWL

Source_z = 0.212 #source redshift

#Telescope
Telescope = "MAGIC" #MAGIC, LST-1 (CTAO-N has no real data yet)
#EBL options
EBL_Model = "dominguez" #options from ebltable
scan_method = 5 # 0 for normal scanning, 1 for scanning and retryng failed fits, 2 for scanning and scanning again with another initial guess, 3 for scanning, retrying failed fits and 
                #scanning again with another initial guess and 4 for doing every alpha by itself (no chained initial guess) Like 3 but doing everything ant taking the best even if it does not fail
alpha_step = 0.05 #EBL scale bin size
alpha_init = 0.0-alpha_step #positon of the first initial guess. used in scan_method 0, 1, 2 and 3   
other_initial_guess_position = 2.0+alpha_step #this is only used for non converging fits in scan_method 2 and 3
alpha_min = 0.0-alpha_step #lowest EBL scale factor
alpha_max = 2.0+alpha_step #hightes EBL scale factor

if Telescope == "MAGIC":
    IRF_u = True
else:
    IRF_u = False

#MBPWL info
Efirst = 0.1694 #First knot position
DeltaE = 0.157 #Distance between knots
knots = 2 #number of knots in the MBPWL

In [ ]:
alphas = alphas_creation(alpha_init, alpha_min, alpha_max, alpha_step)

fit_func = fit_func_select(fit_func_name, knots, Efirst, DeltaE)

if fit_func_name == "MBPWL":
    initial_guess_0 = np.zeros(knots+2)
    initial_guess_0[0] = 5e-6
    initial_guess_0[1] = 2.

elif fit_func_name == "PWL":
    initial_guess_0 = np.array([5e-7, 2.]) #phi_0, gamma

elif fit_func_name == "LP":
    initial_guess_0 = np.array([5e-7, 2., 0.]) #phi_0, alpha, beta

elif fit_func_name == "freeLP":
    initial_guess_0 = np.array([5e-7, 2., 0.]) #phi_0, alpha, beta

elif fit_func_name == "EPWL":
    initial_guess_0 = np.array([5e-6, 2., np.sqrt(3.e4)]) #x[0] = 1.e-9; x[1] = -2.; x[2] = sqrt(3.e4)

elif fit_func_name == "ELP":
    initial_guess_0 = np.array([5e-6, 2., 0.15, np.sqrt(3.e4)]) #1.e-9; x[1] = -2.; x[2] = 0.15; x[3] = sqrt(3.e4)

elif fit_func_name == "SEPWL":
    initial_guess_0 = np.array([5e-6, 2., np.sqrt(3.e4), 1.]) #1.e-9; x[1] = -2.; x[2] = sqrt(3.e4); x[3] = 1.

elif fit_func_name == "SELP":
    initial_guess_0 = np.array([5e-6, 2., 0.15, np.sqrt(3e4), 1.]) #x[0] = 1.e-9; x[1] = -2.; x[2] = 0.15; x[3] = sqrt(3.e4); x[4] = 1.
        
if (Telescope == "MAGIC" or Telescope == "LST-1"):
    Noffregions = 3
    def m2LogL(params):
        xdata = Etrue
        mtau = -tau
        if IRF_u:
            mu_gam, mu_gam_u = dNdE_to_mu_IRF((fit_func(xdata, params) * np.exp(mtau * alpha)), Ebinsw_Etrue, migmatval, migmaterr, Eest)
            if Systematics:
                mu_gam_u = np.sqrt(np.square(mu_gam_u) + np.square(sys_err * mu_gam))
            mu_gam_temp_u = mu_gam_u[minbin:maxbin]

        else:
            mu_gam = dNdE_to_mu((fit_func(xdata, params) * np.exp(mtau * alpha)), Ebinsw_Etrue, migmatval, Eest)

        mu_gam_temp = mu_gam[minbin:maxbin]
        Non_temp = Non[minbin:maxbin] 
        Noff_temp = Noff[minbin:maxbin]
        
        if betterEminmax:
            newminbin, newmaxbin = minbinmaxbinfinal(Non_temp, Noff_temp)
            Non_final = Non_temp[newminbin:newmaxbin]
            Noff_final = Noff_temp[newminbin:newmaxbin]
            mu_gam_final = mu_gam_temp[newminbin:newmaxbin]
            if IRF_u:
                mu_gam_final_u = mu_gam_temp_u[newminbin:newmaxbin]
        else: 
            Non_final = Non_temp
            Noff_final = Noff_temp
            mu_gam_final = mu_gam_temp
            if IRF_u:
                mu_gam_final_u = mu_gam_temp_u

        min_num_gauss = 20

        res = np.ones(len(Non_final)) * 999999999
        for i in range(len(Non_final)):
            if IRF_u:
                if ((Non_final[i] >= min_num_gauss) & (Noff_final[i] >= min_num_gauss)):
                    res[i] = Gauss_logL_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
                elif Non_final[i] == 0:
                    res[i] = Poisson_logL_Non0_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
                elif Noff_final[i] == 0:
                    res[i] = Poisson_logL_Noff0_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
                elif mu_gam_final[i] < 1e-6:
                    res[i] = Poisson_logL_small_mugam_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
                elif mu_gam_final_u[i] == 0:
                    res[i] = Poisson_logL_noIRF_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
                elif (Non_final[i] != 0) & (Noff_final[i] != 0):
                    res[i] = Poisson_logL_else_IRF(Non_final[i], Noff_final[i], mu_gam_final[i], mu_gam_final_u[i], Noffregions)
            
            else:
                if ((Non_final[i] >= min_num_gauss) & (Noff_final[i] >= min_num_gauss)):
                    res[i] = Gauss_logL(Non_final[i], Noff_final[i], mu_gam_final[i], Noffregions)
                elif Non_final[i] == 0:
                    res[i] = Poisson_logL_Non0(Non_final[i], Noff_final[i], mu_gam_final[i], Noffregions)
                elif Noff_final[i] == 0:
                    res[i] = Poisson_logL_Noff0(Non_final[i], Noff_final[i], mu_gam_final[i], Noffregions)
                elif (Non_final[i] != 0) & (Noff_final[i] != 0):
                    res[i] = Poisson_logL_else(Non_final[i], Noff_final[i], mu_gam_final[i], Noffregions) 

        return np.sum(res)

def fit(initial_guess):

    if fit_func_name == "MBPWL":
        if knots == 1:
            names = ("phi0", "gamma0", "Deltag0", "Eknot")
        else:
            names = ("phi0", "gamma0")
            for i in range(knots):
                names = names + ("Deltag{0}".format(i),)
    elif fit_func_name == "PWL":
        names = ("phi0", "gamma0")            
    elif fit_func_name == "LP" or fit_func_name == "freeLP":
        names = ("phi0", "gamma0", "b")
    elif fit_func_name == "EPWL":
        names = ("phi0", "gamma0", "Ecut")
    elif fit_func_name == "ELP":
        names = ("phi0", "gamma0", "b", "Ecut")
    elif fit_func_name == "SEPWL":
        names = ("phi0", "gamma0", "Ecut", "d")
    elif fit_func_name == "SELP":
        names = ("phi0", "gamma0", "b", "Ecut", "d")

    m2LogL.errordef = Minuit.LIKELIHOOD
    m = Minuit(m2LogL, initial_guess, name = names)   

    if fit_func_name == "MBPWL": #defines limits to faster and better find the minimum. Can be changed if the intrinsic spectrum function is changed. 
        MBPWL_limits = ([(1e-6, 1e-3), (-4., 5.)])
        errors = [1e-7, 0.01]
        for i in range(knots):
            MBPWL_limits.append(None)
            errors.append(0.01)
        m.limits = MBPWL_limits
    elif fit_func_name == "PWL":
        m.limits = ([(1e-7,1e-3), None])
        errors = [1e-7, 0.01]
    elif fit_func_name == "LP" or fit_func_name == "freeLP":
        m.limits = ([(1e-7, 1e-3), (None, None), (None, None)])
        errors = [1e-7, 0.01, 0.01]
    elif fit_func_name == "EPWL":
        m.limits = ([(1e-7, 1e-3), (None, None), (None, None)])
        errors = [1e-8, 1e-3, 1e-3]
    elif fit_func_name == "ELP":
        m.limits = ([(1e-7, 1e-3), (-2., None), (None, None), (None, None)])
        errors = [1e-8, 1., 0.1, np.sqrt(500.)]
    elif fit_func_name == "SEPWL":
        m.limits = ([(1e-7, 1e-3), (None, None), (None, None), (None, None)])
        errors = [1e-8, 1.0, np.sqrt(500.), 1.]

    elif fit_func_name == "SELP":
        m.limits = ([(1e-7, 1e-3), (-2., None), (None, None), (None, None), (None, None)])
        errors = [1e-8, 1., 0.1, np.sqrt(500.), 0.1]
    m.errors = errors
     
    m.migrad()
    return m

In [ ]:
if Telescope == "MAGIC": #compute values needed for minimization if the selected telescope is MAGIC
    
    Bckg = uproot.open("{0}:hEstBckgE".format(data_path))#load background values
    bckgmu_final = Bckg.values() #background counts in the data

    migrmatrix = uproot.open("MAGIC/fold_migmatrix_{0}.root:mig_matrix".format(Source_name))
    migmatval = migrmatrix.values() #m^2 * s #values
    if IRF_u:
        migmaterr = migrmatrix.errors()
    migmatxEtrue = migrmatrix.axis("x").edges()/1e3 #TeV #edge values of X axis of the migration matrix (True Energy)
    migmatyEest = migrmatrix.axis("y").edges()/1e3 #TeV #edge values of Y axis of the migration matrix (Estimated Energy)

    Eest = migrmatrix.axis("y").centers()/1e3 #TeV #center values of X axis of the migration matrix (True Energy)
    Etrue = migrmatrix.axis("x").centers()/1e3 #TeV #center values of Y axis of the migration matrix (Estimated Energy)
    Usedbins = np.where((Emin <= Eest) & (Eest <= Emax))
    minbin = Usedbins[0][0]
    maxbin = Usedbins[0][-1] + 1
    Eest_final = Eest[minbin:maxbin]
    
    tau1 =  OptDepth.readmodel(model=EBL_Model)
    tau = tau1.opt_depth(Source_z, Etrue)
    Ebinsw_final = migmatyEest[1:] - migmatyEest[:-1] #compute the bin width of the final energy bins
    Ebinsw_Etrue = migmatxEtrue[1:] - migmatxEtrue[:-1] #compute the bin width of Etrue energy bins

    excess = uproot.open("MAGIC/excess_{0}.root:_px".format(Source_name))
    mu_vec_final = excess.values() #Energy bins = Eest

elif Telescope == "LST-1":
    data_path = "LST-1_migmat_data" #change when you have the real data to analize
    print("Warning: Make sure that the migration matrix folder is the one for your data and thay you got it using LST-1_IRF.ipynb")
    bckgmu_final = np.loadtxt("{0}/bckg.txt".format(data_path))   
    migmatval = np.loadtxt("{0}/migmatval.txt".format(data_path))
    migmatxEtrue = np.loadtxt("{0}/migmatxEtrue.txt".format(data_path))/1e3 #TeV #edge values of X axis of the migration matrix (True Energy)
    migmatyEest = np.loadtxt("{0}/migmatyEest.txt".format(data_path))/1e3 #TeV #edge values of Y axis of the migration matrix (Estimated Energy)
    Eest = np.loadtxt("{0}/Eest.txt".format(data_path))/1e3 #TeV #center values of X axis of the migration matrix (True Energy)
    Etrue = np.loadtxt("{0}/Etrue.txt".format(data_path))/1e3 #TeV #center values of Y axis of the migration matrix (Estimated Energy)

    Usedbins = np.where((Emin <= Eest) & (Eest <= Emax))
    minbin = Usedbins[0][0]
    maxbin = Usedbins[0][-1] + 1
    Eest_final = Eest[minbin:maxbin]

    tau1 =  OptDepth.readmodel(model=EBL_Model)
    tau = tau1.opt_depth(Source_z, Etrue)
    Ebinsw_final = migmatyEest[1:] - migmatyEest[:-1] #compute the bin width of the final energy bins
    Ebinsw_Etrue = migmatxEtrue[1:] - migmatxEtrue[:-1] #compute the bin width of Etrue energy bins


    mu_vec_final = np.loadtxt("{0}/excess.txt".format(data_path)) #Energy bins = Eest

In [ ]:
def process2(alphas, mu_on, mu_off):
    chisqs = []
    chi_bf = 9999999
    chisqs = np.ones(len(alphas)) * 99999999
    global alpha, Non, Noff
    alpha = alpha_init
    Non, Noff = np.round(mu_on), np.round(Noffregions * mu_off)
    bestfit = initial_guess_0
    if scan_method == 4:
        for i, alpha0 in enumerate(alphas):
            alpha = alpha0
            things = fit(initial_guess = initial_guess_0)
            if things.valid == False:
                print("Function {0} did not minimize properly the {1} intrinsic spectra for alpha {2}".format(fit_func_name, Source_name, alpha))
            else:
                chi2 = m2LogL(things.values)
                chisqs[i] = chi2
                if chi_bf > chi2:
                    chi_bf = chi2
                    bestfit = things.values
    else:
        things = fit(initial_guess=initial_guess_0)
        initial_guess_mat = ig_mat_create(fit_func_name, alphas, knots)
        initial_guess_mat[0] = things.values
        for i, alpha0 in enumerate(alphas):
            alpha = alpha0
            initial_guess = initial_guess_mat[i]
            if alpha == alpha_init:
                initial_guess = initial_guess_mat[0]
            things = fit(initial_guess = initial_guess)
            if i < len(alphas):
                initial_guess_mat[i+1] = things.values
            if things.valid == False:
                print("Function {0} did not minimize properly the {1} intrinsic spectra for alpha {2}".format(fit_func_name, Source_name, alpha))
            else:
                chi2 = m2LogL(things.values)
                chisqs[i] = chi2
                if chi_bf > chi2:
                    chi_bf = chi2
                    bestfit = things.values
                if alpha == 1 or alpha == 0.995:
                    print("alpha = 1 best fit: ", things.values)
                    bestfit_alpha1 = things.values
                    chialph1 = chi2

        if scan_method == 1 or scan_method == 3:
            for i, alpha0 in enumerate(reversed(alphas)):
                j = len(alphas) - i - 1
                if chisqs[j] == 99999999:
                    alpha = alpha0
                    if i == 0:
                        initial_guess = initial_guess_mat[0]
                    else:
                        initial_guess = initial_guess_mat[j+2]
                    print("Retrying alpha = {0} with different initial guess: {1}".format(alpha0, initial_guess))
                    things = fit(initial_guess = initial_guess)
                    if things.valid == False:
                        print("Function {0} did not minimize properly with the other initial guess".format(fit_func_name))
                        print("The initial guess was: ", initial_guess)
                        break
                    else:    
                        print("Function {0} minimized properly with the other initial guess".format(fit_func_name))
                        chi2 = m2LogL(things.values)
                        initial_guess_mat[j+1] = things.values
                        # print("chi2: ", chi2)
                        chisqs[j] = chi2 
                        if chi_bf > chi2:
                            chi_bf = chi2
                            bestfit = things.values
                        if alpha == 1 or alpha == 0.995:
                            print("alpha = 1 best fit: ", things.values)
                            bestfit_alpha1 = things.values
                            chialph1 = chi2
        elif scan_method == 5:
            chialph1 = 99999999
            for i, alpha0 in enumerate(reversed(alphas)):
                j = len(alphas) - i - 1
                alpha = alpha0
                if i == 0:
                    initial_guess = initial_guess_mat[0]
                else:
                    initial_guess = initial_guess_mat[j+2]
                things = fit(initial_guess = initial_guess)
                if things.valid == False:
                    print("Function {0} did not minimize properly with the other initial guess for alpha {1}".format(fit_func_name, alpha))
                else:    
                    chi2 = m2LogL(things.values)
                    initial_guess_mat[j+1] = things.values
                    if chi2 < chisqs[j]:
                        chisqs[j] = chi2 
                        if chi_bf > chi2:
                            chi_bf = chi2
                            bestfit = things.values
                        if alpha == 1 or alpha == 0.995:
                            if chi2 < chialph1:
                                print("alpha = 1 best fit: ", things.values)
                                bestfit_alpha1 = things.values
                                chialph1 = chi2

                    
    return chisqs, bestfit, bestfit_alpha1, chialph1

mu_on = mu_vec_final + bckgmu_final
mu_off = bckgmu_final 
chisqs1, bestfit1, bestfit_alpha1_1, chialpha1 = process2(alphas, mu_on, mu_off)
if scan_method == 2 or scan_method == 3:
    if (chisqs1 == 99999999).any() or (chisqs1 == np.inf).any(): #if the first try did not work, we try to minimize it with another initial guess position
        alphas2 = alphas_creation(other_initial_guess_position, alpha_min, alpha_max, alpha_step)
        chisqs2, bestfit2, bestfit_alpha1_2, chialpha2 = process2(alphas2, mu_on, mu_off)

        initial_pos = np.where(alphas == alphas2[0])[0][0]
        indices = np.argsort(alphas)
        index_init = np.where(indices == initial_pos)[0][0]
        chisqs2 = np.concatenate((chisqs2[indices[index_init:]], np.flip(chisqs2[indices[:index_init]])))
        chisqs = np.minimum(chisqs1, chisqs2)
        bestfit_alpha1 = bestfit_alpha1_1 if chialpha1 < chialpha2 else bestfit_alpha1_2
        bestfit = bestfit1 if np.min(chisqs1) < np.min(chisqs2) else bestfit2
    else:
        chisqs = chisqs1
        bestfit = bestfit1

elif scan_method == 5:
    alphas2 = alphas_creation(other_initial_guess_position, alpha_min, alpha_max, alpha_step)
    chisqs2, bestfit2, bestfit_alpha1_2, chialpha2 = process2(alphas2, mu_on, mu_off)


    initial_pos = np.where(alphas == alphas2[0])[0][0]
    indices = np.argsort(alphas)
    index_init = np.where(indices == initial_pos)[0][0]
    chisqs2 = np.concatenate((chisqs2[indices[index_init:]], np.flip(chisqs2[indices[:index_init]])))
    chisqs = np.minimum(chisqs1, chisqs2)
    bestfit_alpha1 = bestfit_alpha1_1 if chialpha1 < chialpha2 else bestfit_alpha1_2
    bestfit = bestfit1 if np.min(chisqs1) < np.min(chisqs2) else bestfit2
             
else:
    chisqs = chisqs1
    bestfit = bestfit1

print(bestfit)
print("bestfit_alpha1: ", bestfit_alpha1)

if not os.path.exists("chi2fits"):
    os.makedirs(f"chi2fits")
hdf5filename = "chi2fits/chi2_{0}_{1}_{2}.hdf5".format(Source_name, fit_func_name, extratxt)
savefile = h5py.File(hdf5filename, "w")
order = np.argsort(alphas)
alphas2 = np.take_along_axis(alphas, order, axis=-1)[1:-1]
chisqs2 = np.take_along_axis(np.array(chisqs), order, axis=-1)[1:-1]
dset = savefile.create_dataset("alphas", data = alphas2, dtype='float')
dset = savefile.create_dataset("chisqs", data = chisqs2, dtype='float')
savefile.close()

## Plot of the data

In [ ]:
#plot of chi2 vs alpha
order = np.argsort(alphas)
alphas2 = np.take_along_axis(alphas, order, axis=-1)[1:-1]
chisqs2 = np.take_along_axis(np.array(chisqs), order, axis=-1)[1:-1]
interpx = np.arange(alpha_min + alpha_step, alpha_max - alpha_step, alpha_step/1000)
f1 = interpolate.interp1d(alphas2, chisqs2, kind='cubic')
chis_new = f1(interpx)
minimum = np.round(interpx[np.where(chis_new == np.min(chis_new))][0], decimals = 3)
sigma_inter_1s = np.where(chis_new <= 1 + np.min(chis_new))
upper_bound = np.round(interpx[np.max(sigma_inter_1s)], decimals = 3)
lower_bound = np.round(interpx[np.min(sigma_inter_1s)], decimals = 3)

plt.plot(alphas2, chisqs2, color = "red", label = "python code")
limits = plt.ylim()
limmin = limits[0]
limmax = limits[1]
maxymin = (np.min(chis_new)-limmin) / (limmax-limmin)
maxymin1 = (np.min(chis_new)+1-limmin) / (limmax-limmin)
plt.axvline(minimum, ymax = maxymin, color = "red", linestyle = "--")
plt.axvline(upper_bound, ymax = maxymin1, color = "red", linestyle = "--")
if lower_bound != 0:
    plt.axvline(lower_bound, ymax = maxymin1, color = "red", linestyle = "--")
plt.xlabel("EBL scale")
plt.ylabel(r"$-2logL$")
plt.grid(linestyle = ":")
plt.xlim(-0.01,None)
plt.title("{0} with {1} fit ".format(Source_name, fit_func_name))
plt.text(x = 0.05, y = np.max(chisqs2) - (np.max(chisqs2)-np.min(chisqs2))/7, s="EBL scale = {0:.3f} (+{1:.3f}) (-{2:.3f})".format(minimum, np.round(upper_bound-minimum, decimals = 3), np.round(minimum-lower_bound, decimals = 3)), color = "red")
plt.text(x = 0.05, y = np.max(chisqs2) - (np.max(chisqs2)-np.min(chisqs2))/5, s = r"minimum $\chi^2$ = {0:.1f}".format(np.round(np.min(chis_new), decimals = 3)), color = "red")
plt.legend()
plt.savefig("chi2fits/chi2_{0}_{1}_{2}.pdf".format(Source_name, fit_func_name, extratxt))
plt.show()
